In [1]:
import sys
import os
sys.path.append(os.path.abspath('..')) # add parent folder to path
import backtrader as bt
import pandas as pd
import matplotlib
matplotlib.use('QT5Agg')


In [2]:
import collections
MAINSIGNALS = collections.OrderedDict(
    (('longshort', bt.SIGNAL_LONGSHORT),
     ('longonly', bt.SIGNAL_LONG),
     ('shortonly', bt.SIGNAL_SHORT),)
)


EXITSIGNALS = {
    'longexit': bt.SIGNAL_LONGEXIT,
    'shortexit': bt.SIGNAL_LONGEXIT,
}


class SMACloseSignal(bt.Indicator):
    lines = ('signal',)
    params = (('period', 30),)

    def __init__(self):
        self.lines.signal = self.data - bt.indicators.SMA(period=self.p.period)


class SMAExitSignal(bt.Indicator):
    lines = ('signal',)
    params = (('p1', 5), ('p2', 30),)

    def __init__(self):
        sma1 = bt.indicators.SMA(period=self.p.p1)
        sma2 = bt.indicators.SMA(period=self.p.p2)
        self.lines.signal = sma1 - sma2

In [3]:
dataname='../data/GC=F_com_disagg.csv'
# data = OHLCAVWithDisaggCoT(dataname=dataname)

cerebro = bt.Cerebro()
df = pd.read_csv(dataname, index_col=0)
df.index = pd.to_datetime(df.index)
# df


In [4]:
from data_feeds.CoT_Dissag import PandasData_more
from strategies.btStrategy import MyStrategy

data = PandasData_more(dataname=df)

# data = OHLCAVWithDisaggCoT(dataname = dataname)

cerebro.addstrategy(MyStrategy)

cerebro.adddata(data)

# cerebro.addstrategy(MyStrategy)
# cerebro.add_signal(bt.SIGNAL_LONG, SMACloseSignal, period = 10)
# cerebro.add_signal(bt.SIGNAL_LONGSHORT, MySignal)

# 初始资金 100,000,000
cerebro.broker.setcash(1e6)
# 佣金，双边各 0.0003
cerebro.broker.setcommission(commission=0.0003)
# 滑点：双边各 0.0001
cerebro.broker.set_slippage_perc(perc=0.0001)

cerebro.addobserver(bt.observers.BuySell) # 买卖交易点

cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='pnl') # 返回收益率时序数据
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn') # 年化收益率
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_SharpeRatio') # 夏普比率
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown') # 回撤


In [5]:
# 启动回测
result = cerebro.run()
cerebro.plot(iplot= False)
# 从返回的 result 中提取回测结果
strat = result[0]
# 返回日度收益率序列
daily_return = pd.Series(strat.analyzers.pnl.get_analysis())
# 打印评价指标
print("--------------- AnnualReturn -----------------")
print(strat.analyzers._AnnualReturn.get_analysis())
print("--------------- SharpeRatio -----------------")
print(strat.analyzers._SharpeRatio.get_analysis())
print("--------------- DrawDown -----------------")
print(strat.analyzers._DrawDown.get_analysis())



init
--------- 打印 self.datas 第一个数据表格的 lines ----------
('close', 'low', 'high', 'open', 'volume', 'openinterest', 'datetime', 'Tot_Rept_Positions_Long_All', 'Tot_Rept_Positions_Short_All', 'M_Money_Positions_Long_All', 'M_Money_Positions_Short_All')
Type of self.sma: <class 'backtrader.indicators.sma.SimpleMovingAverage'>
--------------- AnnualReturn -----------------
OrderedDict([(2019, -0.00011467011223498247), (2020, -0.0011706946182376), (2021, 0.00039410637103864943), (2022, 0.0007189438316088648)])
--------------- SharpeRatio -----------------
OrderedDict([('sharperatio', -14.034080088662387)])
--------------- DrawDown -----------------
AutoOrderedDict([('len', 704), ('drawdown', 0.03002742252710917), ('moneydown', 300.31235544988886), ('max', AutoOrderedDict([('len', 704), ('drawdown', 0.22965081836948079), ('moneydown', 2296.7998046875)]))])
